<a href="https://colab.research.google.com/github/bcolorado/ML-course/blob/main/LAB_02_03_PANDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 02.03 - Pandas

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/fagonzalezo/ai4eng-unal/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


replicating local resources


In [ ]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

logging in as bcolorado@unal.edu.co... please wait
using course session ai4eng.v1.unal::intro_si_2024_1
success!! you are logged in


In [ ]:
import numpy as np
import pandas as pd

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [ ]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)

    if missing:
        nmissing = np.random.randint(len(price)//2)+2
        price[np.random.permutation(len(price))[:nmissing]] = np.nan

    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]

    return d

In [ ]:
d = create_df()
d

,price,category,margin
itemid,,,
85858,110.68,2.0,10.87
84693,90.22,2.0,10.34
92054,103.15,2.0,9.38
6662,84.34,2.0,11.57
74733,100.17,0.0,10.64
62818,93.55,1.0,9.37
39018,86.32,0.0,11.64
99264,100.97,2.0,9.12
82708,89.62,2.0,11.05


In [ ]:
def select_items(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()

    # Filter rows based on conditions
    if 'margin' in df.columns:
        filtered_df = df[(df['price'] > 100) | (df['margin'] > 10)]
    else:
        filtered_df = df[df['price'] > 100]

    # Extract item ids from the filtered dataframe
    result = filtered_df.index.tolist()

    return result

**manually check your answer**

In [ ]:
d = create_df()
d

,price,category
itemid,,
57572,79.96,0.0
11935,110.92,0.0
6357,100.91,0.0
69549,103.27,1.0
87975,90.20,2.0
9072,112.41,1.0
64197,95.51,2.0
19434,101.78,2.0
70524,97.42,1.0


In [ ]:
select_items(d)

[11935, 6357, 69549, 9072, 19434, 34797]

**submit your code**

In [ ]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category.

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [ ]:
def get_stats(df):

    df = df.copy()

    if "margin" in df.columns:
      df = df.drop("margin", axis=1)

    # Calcular estadisticas
    grouped_stats = df.groupby('category').agg(['mean', 'max', 'min'])

    grouped_stats.columns = grouped_stats.columns.droplevel()

    # Renombrar columnas
    grouped_stats.columns = ['media', 'maximo', 'minimo']
    grouped_stats.index.name = 'categoria'

    # Pasar a Int el indice
    grouped_stats.index = grouped_stats.index.astype(int)

    return grouped_stats


**manually check your answer**

In [ ]:
d = create_df()
d

,price,category
itemid,,
88304,103.03,0.0
49224,112.87,2.0
37826,96.96,1.0
91496,93.37,1.0
74343,109.12,1.0
6676,102.48,0.0
80667,106.68,2.0
39939,95.77,2.0
75791,113.08,1.0


In [ ]:
get_stats(d)

,media,maximo,minimo
categoria,,,
0,102.7550,103.03,102.48
1,103.1325,113.08,93.37
2,103.0575,112.87,95.77


**submit your code**

In [ ]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [ ]:
d = create_df(missing=True)
d

,price,category
itemid,,
69381,NaN,0.0
34625,NaN,1.0
50278,NaN,0.0
91888,106.72,2.0
34025,99.90,1.0
99259,103.92,0.0
98310,NaN,2.0
90293,90.82,1.0
46599,NaN,0.0


In [ ]:

def fillna(df):

    df_copy = df.copy()

    mean_price = df_copy['price'].mean()
    std_price = df_copy['price'].std()

    missing_values = df_copy['price'].isna().sum()
    sampled_values = np.random.normal(loc=mean_price, scale=std_price, size=missing_values)

    # Cambiamos valores faltantes por las muestras
    df_copy.loc[df_copy['price'].isna(), 'price'] = sampled_values

    return df_copy


**check manually your code**

In [ ]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
44033,104.38,0.0,10.51
50520,107.66,1.0,8.32
100478,NaN,0.0,9.14
19985,107.19,2.0,12.27
34041,86.11,1.0,11.78
66424,NaN,0.0,8.99
68734,101.21,0.0,8.45
61190,NaN,1.0,10.71
89198,96.35,2.0,10.18


In [ ]:
fillna(d)

,price,category,margin
itemid,,,
44033,104.380000,0.0,10.51
50520,107.660000,1.0,8.32
100478,101.520936,0.0,9.14
19985,107.190000,2.0,12.27
34041,86.110000,1.0,11.78
66424,115.531834,0.0,8.99
68734,101.210000,0.0,8.45
61190,93.931300,1.0,10.71
89198,96.350000,2.0,10.18


**submit your code**

In [ ]:
student.submit_task(globals(), task_id="task_03");